In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Earthquake-priv/Data.xlsx


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use('seaborn-deep')
plt.style.use('fivethirtyeight')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 14
plt.rcParams['figure.figsize'] = (12, 8)

pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 400)
import warnings
warnings.filterwarnings('ignore')
import sklearn.base as skb
import sklearn.metrics as skm
import sklearn.model_selection as skms
import sklearn.preprocessing as skp
import sklearn.utils as sku
import sklearn.linear_model as sklm
import sklearn.neighbors as skn
import sklearn.ensemble as ske
import catboost as cb
import scipy.stats as sstats
import random
seed = 12
np.random.seed(seed)

from datetime import date

import pandas_profiling as pp

In [3]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa


print(tf.__version__)

# Make NumPy printouts easier to read.


2.6.4


In [4]:
# !pip install tensorflow-addons

In [5]:
dataset = pd.read_excel("/kaggle/input/Earthquake-priv/Data.xlsx")

In [6]:
dataset.head()

,PGA,PGV,Ia,CAR,Ky,P_Disp
0,0.04,0.03,0.02,0.46,0.02,0.15
1,0.04,0.03,0.02,0.52,0.02,0.24
2,0.13,0.07,0.10,0.15,0.02,3.33
3,0.10,0.05,0.07,0.21,0.02,1.88
4,0.06,0.04,0.04,0.32,0.02,0.90


In [7]:
# profile = pp.ProfileReport(df_train, title='Pandas Profiling Report', explorative=True)
# profile.to_file("profile.html")

In [8]:
dataset.isna().sum()

PGA       0
PGV       0
Ia        0
CAR       0
Ky        0
P_Disp    0
dtype: int64

In [9]:
dataset.describe()

,PGA,PGV,Ia,CAR,Ky,P_Disp
count,13707.00,13707.00,13707.00,13707.00,13707.00,13707.00
mean,0.22,0.22,0.88,0.45,0.08,14.32
std,0.21,0.24,1.74,0.24,0.08,51.74
min,0.02,0.00,0.00,0.01,0.02,0.01
25%,0.08,0.07,0.10,0.24,0.02,0.19
50%,0.15,0.13,0.28,0.43,0.05,1.20
75%,0.29,0.28,0.87,0.64,0.10,7.34
max,1.78,2.63,22.75,0.97,0.40,1883.85


In [11]:
dataset = dataset.drop(dataset[(dataset.P_Disp > 150) | (dataset.P_Disp < 0.015)].index)
dataset.head()

,PGA,PGV,Ia,CAR,Ky,P_Disp
0,0.04,0.03,0.02,0.46,0.02,0.15
1,0.04,0.03,0.02,0.52,0.02,0.24
2,0.13,0.07,0.10,0.15,0.02,3.33
3,0.10,0.05,0.07,0.21,0.02,1.88
4,0.06,0.04,0.04,0.32,0.02,0.90


In [12]:
# sns.pairplot(dataset[['PGA', 'PGV', 'Ia', 'CAR','Ky','P_Disp' ]], diag_kind='kde')

In [13]:
dataset = np.log(dataset)
dataset.head()

,PGA,PGV,Ia,CAR,Ky,P_Disp
0,-3.13,-3.66,-4.17,-0.78,-3.91,-1.87
1,-3.27,-3.45,-3.88,-0.65,-3.91,-1.41
2,-2.01,-2.63,-2.29,-1.90,-3.91,1.20
3,-2.34,-2.91,-2.59,-1.57,-3.91,0.63
4,-2.78,-3.31,-3.27,-1.13,-3.91,-0.11


In [14]:
# np.exp(-3.13)

In [15]:
# sns.histplot(dataset.PGA)

In [16]:
# sns.histplot(dataset.PGV)

In [17]:
# sns.histplot(dataset.Ia)

In [18]:
# sns.histplot(dataset.CAR)

In [19]:
# sns.histplot(dataset.Ky)

In [20]:
# sns.histplot(dataset.P_Disp)


In [21]:
# sns.pairplot(dataset[['PGA', 'PGV', 'Ia', 'CAR','Ky','P_Disp' ]], diag_kind='kde')

In [22]:
from sklearn.model_selection import train_test_split

X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]
 
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)


In [23]:
train_features = X_train
test_features = X_test

train_labels = y_train
test_labels = y_test

In [24]:
train_features.head()

,PGA,PGV,Ia,CAR,Ky
11124,-1.38,-0.72,-0.14,-0.52,-1.90
9323,-2.02,-1.62,-1.44,-0.57,-2.59
7337,-2.31,-2.82,-1.66,-0.69,-3.00
1802,-2.25,-2.51,-2.56,-1.67,-3.91
7883,-2.35,-3.10,-3.51,-0.24,-2.59


In [25]:
# normalizer = tf.keras.layers.Normalization(axis=-1)
# normalizer.adapt(np.array(train_features))
# print(normalizer.mean.numpy())

In [26]:
# input_data  = np.array([[4 , 5 , 6 , 7, 8]])
# normalizer(input_data)

In [50]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.plot(history.history['Std_err'], label='Std_err')
  plt.ylim([0, 5])
  plt.xlabel('Epoch')
  plt.ylabel('Error [P_Disp]')
  plt.legend()
  plt.grid(True)

In [28]:
from keras import backend as k
# def matthews_correlation(y_true, y_pred):
#     y_pred_pos = K.round(K.clip(y_pred, 0, 1))
#     y_pred_neg = 1 - y_pred_pos

#     y_pos = K.round(K.clip(y_true, 0, 1))
#     y_neg = 1 - y_pos

#     tp = K.sum(y_pos * y_pred_pos)
#     tn = K.sum(y_neg * y_pred_neg)

#     fp = K.sum(y_neg * y_pred_pos)
#     fn = K.sum(y_pos * y_pred_neg)

#     numerator = (tp * tn - fp * fn)
#     denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

#     return numerator / (denominator + K.epsilon())

def Std_err(y_true, y_pred):
    err  = y_pred - y_true
    mean_err = k.mean(err)
    d = err - mean_err
    ans = k.sqrt(k.mean(k.sum(k.square(d))))
    return ans


                
    


In [29]:
def Corr_R(y_true, y_pred):
    y_true_mean = k.mean(y_true)
    y_pred_mean = k.mean(y_pred)
    num = k.sum((y_true-y_true_mean)*(y_pred-y_pred_mean))
    den = k.sqrt((k.sum(k.square(y_true-y_true_mean)))*(k.sum(k.square(y_pred-y_pred_mean))))
    return num/den

In [30]:
def build_and_compile_model(n):
  model = keras.Sequential([
#       norm,
      layers.Dense(8, input_shape=(n,)),
      layers.Dense(8, activation='relu'),
#       layers.Dense(8, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001), metrics = ['MeanAbsoluteError',Std_err,Corr_R])
  return model

In [31]:
dnn_model = build_and_compile_model(5)
dnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 48        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________


2022-10-10 07:50:44.331152: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-10 07:50:44.427746: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-10 07:50:44.428548: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-10 07:50:44.429644: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [32]:
%%time
history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=1, epochs=25)

2022-10-10 07:50:47.120753: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25
263/263 [==============================] - 2s 4ms/step - loss: 1.3670 - mean_absolute_error: 1.3670 - Std_err: 9.0078 - Corr_R: 0.5887 - val_loss: 0.7717 - val_mean_absolute_error: 0.7717 - val_Std_err: 5.3977 - val_Corr_R: 0.9097
Epoch 2/25
263/263 [==============================] - 1s 3ms/step - loss: 0.6987 - mean_absolute_error: 0.6987 - Std_err: 5.1539 - Corr_R: 0.9104 - val_loss: 0.6381 - val_mean_absolute_error: 0.6381 - val_Std_err: 4.5730 - val_Corr_R: 0.9317
Epoch 3/25
263/263 [==============================] - 1s 4ms/step - loss: 0.6525 - mean_absolute_error: 0.6525 - Std_err: 4.7881 - Corr_R: 0.9239 - val_loss: 0.6264 - val_mean_absolute_error: 0.6264 - val_Std_err: 4.4638 - val_Corr_R: 0.9348
Epoch 4/25
263/263 [==============================] - 1s 3ms/step - loss: 0.6412 - mean_absolute_error: 0.6412 - Std_err: 4.7021 - Corr_R: 0.9260 - val_loss: 0.6223 - val_mean_absolute_error: 0.6223 - val_Std_err: 4.3957 - val_Corr_R: 0.9359
Epoch 5/25
263/263 [============

In [33]:
plot_loss(history)

In [34]:
# test_results =[]
# test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)

In [35]:
input_data = np.log(np.array([[0.0621619,0.036424837,0.038161434,0.321740487,0.02]]))
print(input_data)
# input  = normalizer(input_data)
# print(input)
t = np.exp(dnn_model.predict(input_data))
t

[[-2.77801301 -3.3125044  -3.26592985 -1.13401    -3.91202301]]


array([[1.0344168]], dtype=float32)

PGA, PGV, Ia

----------------------------------------------------------------------------------------------------------------------------------------________________________________________________________________________________________________________________________________________

In [36]:
from sklearn.model_selection import train_test_split

X = dataset[['PGA','PGV','Ia']]
y = dataset.iloc[:, -1]
 
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

In [37]:
X.head()

,PGA,PGV,Ia
0,-3.13,-3.66,-4.17
1,-3.27,-3.45,-3.88
2,-2.01,-2.63,-2.29
3,-2.34,-2.91,-2.59
4,-2.78,-3.31,-3.27


In [38]:
dnn_model = build_and_compile_model(3)
dnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 32        
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


In [39]:
%%time
history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=1, epochs=25)

Epoch 1/25
263/263 [==============================] - 1s 3ms/step - loss: 1.6051 - mean_absolute_error: 1.6051 - Std_err: 10.6949 - Corr_R: 0.5282 - val_loss: 1.5308 - val_mean_absolute_error: 1.5308 - val_Std_err: 10.2370 - val_Corr_R: 0.5712
Epoch 2/25
263/263 [==============================] - 1s 3ms/step - loss: 1.5324 - mean_absolute_error: 1.5324 - Std_err: 10.4064 - Corr_R: 0.5493 - val_loss: 1.5155 - val_mean_absolute_error: 1.5155 - val_Std_err: 10.1003 - val_Corr_R: 0.5865
Epoch 3/25
263/263 [==============================] - 1s 3ms/step - loss: 1.5189 - mean_absolute_error: 1.5189 - Std_err: 10.3251 - Corr_R: 0.5559 - val_loss: 1.5060 - val_mean_absolute_error: 1.5060 - val_Std_err: 10.0730 - val_Corr_R: 0.5918
Epoch 4/25
263/263 [==============================] - 1s 3ms/step - loss: 1.5151 - mean_absolute_error: 1.5151 - Std_err: 10.3029 - Corr_R: 0.5619 - val_loss: 1.5032 - val_mean_absolute_error: 1.5032 - val_Std_err: 10.0403 - val_Corr_R: 0.5936
Epoch 5/25
263/263 [====

In [40]:
from sklearn.model_selection import train_test_split

X = dataset[['PGA','PGV','CAR']]
y = dataset.iloc[:, -1]
 
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

dnn_model = build_and_compile_model(3)
dnn_model.summary()


history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=1, epochs=25)

plot_loss(history)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 8)                 32        
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 9         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
263/263 [==============================] - 1s 3ms/step - loss: 1.4065 - mean_absolute_error: 1.4065 - Std_err: 9.3725 - Corr_R: 0.7028 - val_loss: 0.7301 - val_mean_absolute_error: 0.7301 - val_Std_err: 5.0949 - val_Corr_R: 0.9159
Epoch 2/25
263/263 [==============================] - 1s 3ms/step - loss: 0.6787 - mean_absolute_error: 0.6787 - Std_err: 4.7977 - Corr_R: 0.922

In [41]:
from sklearn.model_selection import train_test_split

X = dataset[['PGA','PGV','Ky']]
y = dataset.iloc[:, -1]
 
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

dnn_model = build_and_compile_model(3)
dnn_model.summary()


history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=1, epochs=25)

plot_loss(history)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 8)                 32        
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 9         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
263/263 [==============================] - 1s 3ms/step - loss: 1.2792 - mean_absolute_error: 1.2792 - Std_err: 8.5537 - Corr_R: 0.8281 - val_loss: 0.7921 - val_mean_absolute_error: 0.7921 - val_Std_err: 5.5029 - val_Corr_R: 0.9041
Epoch 2/25
263/263 [==============================] - 1s 3ms/step - loss: 0.6981 - mean_absolute_error: 0.6981 - Std_err: 4.9676 - Corr_R: 0.918

In [42]:
dataset.head()

,PGA,PGV,Ia,CAR,Ky,P_Disp
0,-3.13,-3.66,-4.17,-0.78,-3.91,-1.87
1,-3.27,-3.45,-3.88,-0.65,-3.91,-1.41
2,-2.01,-2.63,-2.29,-1.90,-3.91,1.20
3,-2.34,-2.91,-2.59,-1.57,-3.91,0.63
4,-2.78,-3.31,-3.27,-1.13,-3.91,-0.11


In [43]:
X_train.head()

,PGA,PGV,Ky
11124,-1.38,-0.72,-1.90
9323,-2.02,-1.62,-2.59
7337,-2.31,-2.82,-3.00
1802,-2.25,-2.51,-3.91
7883,-2.35,-3.10,-2.59


In [46]:
from sklearn.model_selection import train_test_split

X = dataset[['PGA','PGV','Ia','CAR']]
y = dataset.iloc[:, -1]
 
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

dnn_model = build_and_compile_model(4)
dnn_model.summary()


history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=1, epochs=25)

plot_loss(history)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 8)                 40        
_________________________________________________________________
dense_16 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 9         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
263/263 [==============================] - 1s 3ms/step - loss: 1.1367 - mean_absolute_error: 1.1367 - Std_err: 7.6349 - Corr_R: 0.8518 - val_loss: 0.7437 - val_mean_absolute_error: 0.7437 - val_Std_err: 5.2006 - val_Corr_R: 0.9092
Epoch 2/25
263/263 [==============================] - 1s 3ms/step - loss: 0.7099 - mean_absolute_error: 0.7099 - Std_err: 5.0279 - Corr_R: 0.915

In [52]:
plot_loss(history)